In [1]:
module O

using Distributions
using LinearAlgebra
using Random
using Printf

function pvalue_N(X)
    n = length(X)
    z = sum(X) / √n
    ccdf(Chisq(1), z^2)
end

function pvalue_F(X)
    n = length(X)
    χ² = dot(X, X)
    ccdf(Chisq(n), χ²)
end

struct SimPval{M,P,T}
    μ::M
    pval_N::P
    pval_F::P
    α::T
end

function Base.show(io::IO, ::MIME"text/plain", sp::SimPval)
    (; μ, pval_N, pval_F, α) = sp
    Nrej = pval_N .< α
    Nacc = .!Nrej
    Frej = pval_F .< α
    Facc = .!Frej
    if iszero(mean(μ))
        print(io, @sprintf("alpha error rate for test of \"mean(μ)=0\": %5.1f%%\n", 100mean(Nrej)))
    else
        print(io, @sprintf("power for test of \"mean(μ)=0\": %5.1f%%    (mean(μ) = %.3f)\n", 100mean(Nrej), mean(μ)))
    end
    if iszero(μ)
        print(io, @sprintf("alpha error rate for test of \"norm(μ)=0\": %5.1f%%\n", 100mean(Frej)))
    else
        print(io, @sprintf("power for test of \"norm(μ)=0\": %5.1f%%    (norm(μ) = %.3f)\n", 100mean(Frej), norm(μ)))
    end
    print(io, "\n")
    print(io, @sprintf("                       %18s  %22s\n", "reject \"norm(μ)=0\"", "not reject \"norm(μ)=0\""))
    print(io, @sprintf("reject     \"mean(μ)=0\" %11.1f%%  %18.1f%%\n", 100mean(Nrej .& Frej), 100mean(Nrej .& Facc)))
    print(io, @sprintf("not reject \"mean(μ)=0\" %11.1f%%  %18.1f%%\n", 100mean(Nacc .& Frej), 100mean(Nacc .& Facc)))
end

function sim_pval(; n=20, μ=zeros(n), niters=10^6, α=0.05)
    pval_N = zeros(niters)
    pval_F = zeros(niters)
    dist = product_distribution(Normal.(μ))
    Xtmp = [rand(dist) for _ in 1:Threads.nthreads()]
    Threads.@threads :static for i in 1:niters
        tid = Threads.threadid()
        X = rand!(dist, Xtmp[tid])
        pval_N[i] = pvalue_N(X)
        pval_F[i] = pvalue_F(X)
    end
    O.SimPval(μ, pval_N, pval_F, α)
end

end

Main.O

モデル: 
$
X_i \sim \mathrm{Normal}(\mu_i, 1) \quad \text{($i=1,2,\ldots,n$) かつ $X_i$達は独立}
$

帰無仮説N: 
$
\dfrac{\mu_1+\cdots+\mu_n}{n} = 0
$

帰無仮説F: 
$
(\mu_1,\ldots,\mu_n)=(0,\ldots,0)
$

In [2]:
O.sim_pval(; n=5, niters=10^7, α=0.05)

alpha error rate for test of "mean(μ)=0":   5.0%
alpha error rate for test of "norm(μ)=0":   5.0%

                       reject "norm(μ)=0"  not reject "norm(μ)=0"
reject     "mean(μ)=0"         1.4%                 3.6%
not reject "mean(μ)=0"         3.6%                91.4%


In [3]:
O.sim_pval(; n=20, niters=10^7, α=0.05)

alpha error rate for test of "mean(μ)=0":   5.0%
alpha error rate for test of "norm(μ)=0":   5.0%

                       reject "norm(μ)=0"  not reject "norm(μ)=0"
reject     "mean(μ)=0"         0.7%                 4.3%
not reject "mean(μ)=0"         4.3%                90.7%


In [4]:
O.sim_pval(; n=100, niters=10^7, α=0.05)

alpha error rate for test of "mean(μ)=0":   5.0%
alpha error rate for test of "norm(μ)=0":   5.0%

                       reject "norm(μ)=0"  not reject "norm(μ)=0"
reject     "mean(μ)=0"         0.4%                 4.6%
not reject "mean(μ)=0"         4.6%                90.4%


In [5]:
O.sim_pval(; n=1000, niters=10^7, α=0.05)

alpha error rate for test of "mean(μ)=0":   5.0%
alpha error rate for test of "norm(μ)=0":   5.0%

                       reject "norm(μ)=0"  not reject "norm(μ)=0"
reject     "mean(μ)=0"         0.3%                 4.7%
not reject "mean(μ)=0"         4.7%                90.3%


In [6]:
O.sim_pval(; μ=fill(0.5, 20), niters=10^7, α=0.05)

power for test of "mean(μ)=0":  60.9%    (mean(μ) = 0.500)
power for test of "norm(μ)=0":  19.3%    (norm(μ) = 2.236)

                       reject "norm(μ)=0"  not reject "norm(μ)=0"
reject     "mean(μ)=0"        17.0%                43.9%
not reject "mean(μ)=0"         2.3%                36.7%


In [7]:
O.sim_pval(; μ=[fill(0.5, 10); fill(-0.5, 10)], niters=10^7, α=0.05)

alpha error rate for test of "mean(μ)=0":   5.0%
power for test of "norm(μ)=0":  19.3%    (norm(μ) = 2.236)

                       reject "norm(μ)=0"  not reject "norm(μ)=0"
reject     "mean(μ)=0"         1.9%                 3.1%
not reject "mean(μ)=0"        17.5%                77.5%


In [8]:
O.sim_pval(; μ=[5; zeros(19)], niters=10^7, α=0.05)

power for test of "mean(μ)=0":  20.1%    (mean(μ) = 0.250)
power for test of "norm(μ)=0":  88.3%    (norm(μ) = 5.000)

                       reject "norm(μ)=0"  not reject "norm(μ)=0"
reject     "mean(μ)=0"        19.1%                 1.0%
not reject "mean(μ)=0"        69.1%                10.8%
